In [1]:
import numpy as np
import pandas as pd
import glob
import os
import shutil

In [2]:
strClinicalInformationPath = "./tcga_pan_cancer/"

In [7]:
for cancers in os.listdir(strClinicalInformationPath):
    cancer = cancers.split('_')[0]
    if os.path.isdir(f'{strClinicalInformationPath}{cancer}_tcga_pan_can_atlas_2018/'):
        clinicalPath = f'{strClinicalInformationPath}{cancer}_tcga_pan_can_atlas_2018/clinical_data.tsv'
        dfClinicalInformation = pd.read_csv(clinicalPath, sep='\t')
        dfPatientClinical = dfClinicalInformation[
            [
                "Patient ID",
                "Diagnosis Age",
                "Neoplasm Disease Stage American Joint Committee on Cancer Code",
                "Race Category",
                "Sex",
            ]
        ]
        for types in os.listdir(f'{strClinicalInformationPath}{cancer}_tcga_pan_can_atlas_2018/'):
            if types == 'Common Genes':
                geneType = 'Common Genes'
            elif types == 'Targeted Drugs for Genes':
                geneType = 'Mutated Genes'
            if os.path.isdir(f'{strClinicalInformationPath}{cancer}_tcga_pan_can_atlas_2018/{types}/'):
                for name in os.listdir(f'{strClinicalInformationPath}{cancer}_tcga_pan_can_atlas_2018/{types}/'):
                    geneName = name.split('_')[1:]
                    geneName = '_'.join(geneName)
                    dfLabel = pd.read_csv(
                        f"{strClinicalInformationPath}{cancer}_tcga_pan_can_atlas_2018/{types}/{name}/Type of Genetic Alterations Across All Samples.csv"
                    )
                    dfLabel = dfLabel[["Patient ID", "Altered"]]
                    dfLabel.rename(columns={"Altered": f'{geneName}'}, inplace=True)
                    dfPatientClinical = pd.merge(dfPatientClinical, dfLabel, on="Patient ID", how="left")

        dfPatientClinical.to_csv(
            f"{strClinicalInformationPath}{cancer}_tcga_pan_can_atlas_2018/patient_clinical.csv",
            index=False,
        )